# Mastodon API testing

Trying out the API for my local server `infosec.exchange`

Install the library. Docs are here: https://mastodonpy.readthedocs.io/en/stable/index.html

In [ ]:
%pip install Mastodon.py

Import the libraries

In [2]:
import os
import json
import datetime
from mastodon import Mastodon
from mastodon import StreamListener

Get this data from the `Development` tab of the Mastodon website after creating a app.

In [3]:
CONFIG = json.load(open('config.json'))

Instantiate a client.

In [4]:
mastodon = Mastodon(
    access_token = CONFIG['access_token'],
    api_base_url='https://infosec.exchange'
)


check it out!

In [5]:
mastodon

OK, let's get some posts.

In [6]:
timeline = mastodon.timeline_home()

In [7]:
type(timeline)

mastodon.utility.AttribAccessList

In [8]:
timeline[0]

{'id': 109527511262366137,
 'created_at': datetime.datetime(2022, 12, 17, 6, 6, 28, tzinfo=tzutc()),
 'in_reply_to_id': None,
 'in_reply_to_account_id': None,
 'sensitive': False,
 'spoiler_text': '',
 'visibility': 'public',
 'language': None,
 'uri': 'https://ottawa.place/users/leigh/statuses/109527511093383303/activity',
 'url': None,
 'replies_count': 0,
 'reblogs_count': 0,
 'favourites_count': 0,
 'edited_at': None,
 'favourited': False,
 'reblogged': False,
 'muted': False,
 'bookmarked': False,
 'local_only': None,
 'content': '',
 'filtered': [],
 'reblog': {'id': 109526609098714708,
  'created_at': datetime.datetime(2022, 12, 17, 2, 16, 57, tzinfo=tzutc()),
  'in_reply_to_id': None,
  'in_reply_to_account_id': None,
  'sensitive': False,
  'spoiler_text': '',
  'visibility': 'public',
  'language': 'en',
  'uri': 'https://writing.exchange/users/XanIndigo/statuses/109526608610838279',
  'url': 'https://writing.exchange/@XanIndigo/109526608610838279',
  'replies_count': 19,
  '

# Streaming data

This part requires overriding a Listener.

In [9]:
class Listener(StreamListener):
    def __init__(self, client: Mastodon, token: str) -> None:
        self.client = client
        self.token = token
    def on_update(self, status: dict) -> None:
        try:
            status_id = status['id']
            status['created_at'] = status['created_at'].isoformat()
            status['account']['created_at'] = status['account']['created_at'].isoformat()
            status['account']['last_status_at'] = status['account']['last_status_at'].isoformat()
            account_domain = status['account']['acct'].split('@')[1]
            username = status['account']['username']
            os.makedirs(f'data/{account_domain}/{username}',exist_ok=True)
            open(f'data/{account_domain}/{username}/{status_id}.json','w+').write(json.dumps(status,indent=4))
        except:
            print(status)

Set up the listener instance

In [10]:
listener = Listener(mastodon, CONFIG['access_token'])

let it rip!

In [ ]:
mastodon.stream_public(listener)

woo, output in the `data` directory.... let's try something similar on the local server

In [12]:
mastodon.stream_local(listener)

{'id': 109527755105338540, 'created_at': '2022-12-17T07:08:31.641000+00:00', 'in_reply_to_id': None, 'in_reply_to_account_id': None, 'sensitive': False, 'spoiler_text': '', 'visibility': 'public', 'language': 'en', 'uri': 'https://infosec.exchange/users/coleens_/statuses/109527755105338540', 'url': 'https://infosec.exchange/@coleens_/109527755105338540', 'replies_count': 0, 'reblogs_count': 0, 'favourites_count': 0, 'edited_at': None, 'local_only': False, 'content': '<p>damn</p><p><a href="https://youtu.be/DnyF26F7xrc" target="_blank" rel="nofollow noopener noreferrer"><span class="invisible">https://</span><span class="">youtu.be/DnyF26F7xrc</span><span class="invisible"></span></a></p>', 'reblog': None, 'account': {'id': 109304302165280929, 'username': 'coleens_', 'acct': 'coleens_', 'display_name': 'Coleen', 'locked': False, 'bot': False, 'discoverable': True, 'group': False, 'created_at': '2022-11-07T00:00:00+00:00', 'note': '<p>neutral good ☥ sapphic ☥ ventrue ☥ bikini rebel ☥ eng

KeyboardInterrupt: 

Errored out somewhere. perhaps some weird encoding?